# Notebook for the paper titled "Shallow Planning under Partial Observability"

In [ ]:
# Install required packages
%pip install gymnasium
%pip install numpy
%pip install matplotlib
%pip install pymdptoolbox
%pip install joblib
%pip install stable-baselines3

# Import necessary libraries
from typing import Any, Callable, Dict, List, Optional, Tuple, Type, Union
import gymnasium as gym
import numpy as np
from stable_baselines3 import DQN, PPO
from gymnasium import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import SubprocVecEnv
import gc
import matplotlib.pyplot as plt

In [ ]:
def linear_schedule(initial_value: Union[float, str]) -> Callable[[float], float]:
    """
    Linear learning rate schedule.

    :param initial_value: (float or str)
    :return: (function)
    """
    # Force conversion to float
    initial_value_ = float(initial_value)

    def func(progress_remaining: float) -> float:
        """
        Progress will decrease from 1 (beginning) to 0
        :param progress_remaining: (float)
        :return: (float)
        """
        return progress_remaining * initial_value_

    return func

In [ ]:
# Wrapper for the CartPole environment to add noise
class NoisyCartPoleEnv(gym.Wrapper):
    def __init__(self, env, noise_std=0.1):
        super(NoisyCartPoleEnv, self).__init__(env)
        self.noise_std = noise_std

    def reset(self, **kwargs):
        state, info = self.env.reset(**kwargs)
        noisy_state = self.add_noise(state)
        return noisy_state, info

    def step(self, action):
        next_state, reward, terminated, truncated, info = self.env.step(action)
        noisy_next_state = self.add_noise(next_state)
        return noisy_next_state, reward, terminated, truncated, info

    def add_noise(self, state):
        if self.noise_std > 0:
            noise = np.random.normal(0, self.noise_std, size=state.shape)
            return state + noise
        return state

def make_noisy_env(env_id, rank, seed=0, noise_std=0.1):
    def _init():
        env = gym.make(env_id)
        env = NoisyCartPoleEnv(env, noise_std)
        env.reset(seed=seed + rank)
        return env
    return _init

In [ ]:
def play_environment(model, seed,std, max_steps=1000):
    '''
    q_values : (n, n, k) numpy array where n is the number of (discrete) states and k is the number of actions
    seed : int, seed for the random number generator
    '''
    env = gym.make("CartPole-v1")
    env = NoisyCartPoleEnv(env, std)

    time_step = 0
    terminated, truncated = False, False
    obs, _ = env.reset(seed=seed)
    total_reward = 0  # Initialize total reward
    
    while not (terminated or truncated) and time_step < max_steps:
        env.render()
        action, _states = model.predict(obs)
        obs, reward, terminated, truncated, _ = env.step(action)            
        total_reward += reward  # Accumulate the reward
        time_step += 1
    
    env.close()
    return total_reward  # Return the total reward

In [ ]:
#Main loop for training the models
stds = [0.0,0.1,1]
gammas = np.linspace(0, 0.98, 5)

learning_rate_schedule = linear_schedule(1e-3)
clip_range_schedule = linear_schedule(0.2)
n_simuls = 10
for simul in range(n_simuls):
    for i, std in enumerate(stds):
        for j, gamma in enumerate(gammas):
            print("doing std" + str(std) + "with gamma:" + str(gamma)+ "simluation: " + str(simul))

            # Number of parallel environments
            n_envs = 8
            env_id = "CartPole-v1"

            # Create the vectorized environment
            env = SubprocVecEnv([make_noisy_env(env_id, i, noise_std=std) for i in range(n_envs)])

            # Initialize the PPO model with the vectorized environment
            model = PPO(
                policy='MlpPolicy',
                env=env,
                n_steps=32,
                batch_size=256,
                gamma=gamma,
                gae_lambda=0.8,
                n_epochs=20,
                ent_coef=0.0,
                learning_rate=linear_schedule(1e-3),
                clip_range=linear_schedule(0.2),
                verbose=0
            )

            # Learn with the specified number of timesteps
            model.learn(total_timesteps=100000, log_interval=10)
            model.save(f"models/simul{simul}/CartpoleGamma{round(gamma, 2)}std{std}")
            del model
            env.close()
            del env
            gc.collect()

In [ ]:
# Load all the models into memory
stds = [0.0,0.1,1]
gammas = np.linspace(0, 0.98, 5)  # Example values for gamma
n_simuls = 10
import numpy as np
loaded_models = np.empty((n_simuls, len(stds), len(gammas)), dtype=object)

for simul in range(n_simuls):
    for i, std in enumerate(stds):
        for j, gamma in enumerate(gammas):
            model_path = f"models/simul{simul}/CartpoleGamma{round(gamma, 2)}std{std}"
            loaded_models[simul, i, j] = PPO.load(model_path)


In [ ]:
#Print the average reward for each model depending on the noise and gamma
seeds = range(100)
rewards = np.zeros((n_simuls, len(stds), len(gammas), len(seeds)))

def get_reward(model, std, seed):
    return play_environment(model, seed, std, 500)

for seed in seeds:
    for simul in range(n_simuls):
        for i, std in enumerate(stds):
            for j, gamma in enumerate(gammas):
                print("doing simul: " + str(simul) + "with std: " + str(std) + "and gamma: " + str(gamma) + "seed: " + str(seed))
                rewards[simul, i, j, seed] = get_reward(loaded_models[simul, i, j], std, seed)

# Reshape the results back into the original 4D array shape
rewards = np.array(rewards).reshape(n_simuls, len(stds), len(gammas), len(seeds))

# Calculate mean across seeds, then across POMDPs and simulations
mean_rewards = rewards.mean(axis=3)  # Mean across seeds
mean_rewards = mean_rewards.mean(axis=0)  # Mean across simulations

# Calculate mean and std deviation across seeds, then across POMDPs and simulations
std_rewards = rewards.std(axis=3)  # Std deviation across seeds
std_rewards = std_rewards.mean(axis=0)  # Mean across simulations

fig1, ax1 = plt.subplots(figsize=(5.5, 4))
x = np.arange(len(gammas))
markers = ['o', 's', 'D', 'x']  # Circle, square, diamond, star, x

for i, std in enumerate(stds):
    ax1.errorbar(x, mean_rewards[i], yerr=std_rewards[i], label=f'std={std}', marker=markers[i % len(markers)], markersize=8, capsize=5)

ax1.set_xlabel('$\gamma$', fontsize=10)
ax1.set_ylabel('Average Reward', fontsize=10)
ax1.set_xticks(x)
ax1.set_xticklabels([f'{g:.2f}' for g in gammas], fontsize=10)
ax1.legend(fontsize=10)

# Save the plot as a PDF file
plt.savefig('average_reward_plot.pdf')

plt.show()

# Print the average rewards for gamma = 1 (last column) for reference
print("Average rewards for gamma = 1 (across all simulations):")
for i, std in enumerate(stds):
    print(f"std={std}: {mean_rewards[i, -1]:.2f}")